In [1]:
# Mounting Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import pickle
import nltk 

co_occurence_matrix = {}
vocabulary = []
local_context_window = 6
print("Global variables initialized...!!")

with open("/content/drive/My Drive/ISI-Project/train_data_bow","rb") as f:
    corpus = pickle.load(f)

training_data = []

vocabulary.clear()
co_occurence_matrix.clear()
no_of_sentences_so_far = 0 

for poem in corpus:
  poem = poem[-2]

  no_of_sentences_so_far += 1
  if no_of_sentences_so_far%5000 == 0 :
    print("Co occurence matrix build in progress... " + str(no_of_sentences_so_far) + " poems read.")
    
  for index_tokens_list in range(len(poem)):
    token = poem[index_tokens_list]
    if token not in vocabulary:
      vocabulary.append(token)  

    for index in range(1, int(local_context_window/2)):
      if index + index_tokens_list >= len(poem):
        break
      tuple_main_context = (poem[index_tokens_list], poem[index_tokens_list + index])
      tuple_context_main = (poem[index_tokens_list + index], poem[index_tokens_list])
      if tuple_main_context not in co_occurence_matrix:
        co_occurence_matrix[tuple_main_context] = 1
        co_occurence_matrix[tuple_context_main] = 1
      
      else:
        co_occurence_matrix[tuple_main_context] += 1
        co_occurence_matrix[tuple_context_main] += 1

print( "Co occurence matrix built successfully..!!")
print( "Number of words in vocabulary: "  + str(len(vocabulary)))
print( "Number of entries in co-occurence matrix: "  + str(len(co_occurence_matrix)))
print( "Saving the co occurence matrix and vocabulary for future reference.")

pickle.dump( vocabulary, open( "/content/drive/My Drive/ISI_PROJECT/vocabulary", "wb" ) )
pickle.dump( co_occurence_matrix, open( "/content/drive/My Drive/ISI_PROJECT/co_occurence_matrix", "wb" ) )

print("Executed Successfully")

Global variables initialized...!!
Co occurence matrix build in progress... 5000 poems read.
Co occurence matrix build in progress... 10000 poems read.
Co occurence matrix build in progress... 15000 poems read.
Co occurence matrix build in progress... 20000 poems read.
Co occurence matrix build in progress... 25000 poems read.
Co occurence matrix build in progress... 30000 poems read.
Co occurence matrix build in progress... 35000 poems read.
Co occurence matrix build in progress... 40000 poems read.
Co occurence matrix build in progress... 45000 poems read.
Co occurence matrix built successfully..!!
Number of words in vocabulary: 24281
Number of entries in co-occurence matrix: 6447730
Saving the co occurence matrix and vocabulary for future reference.
Executed Successfully


In [24]:
import random
import numpy as np
vectors_main_word = {}
vectors_context_word = {}
biases_main_word = {}
biases_context_word = {}
alpha_glove_model = 0.75
x_max_glove_model = 100

number_of_iterations = 100
learning_rate = 0.001

def find_weight( main_token, context_token ):
  if (context_token,main_token) not in co_occurence_matrix:
    return 0
  if ( co_occurence_matrix[(context_token,main_token)] < x_max_glove_model ):
    return (co_occurence_matrix[(context_token,main_token)] / x_max_glove_model) ** alpha_glove_model
  return 1

def initilize_word_vectors_and_biases():
  for token in vocabulary:
    vectors_main_word[token] = np.random.random(100)
    vectors_context_word[token] = np.random.random(100)
    biases_main_word[token] = random.random()
    biases_context_word[token] = random.random()

initilize_word_vectors_and_biases()

print("Initialization of the word vectors and biases for the " + str(len(vocabulary)) + " tokens in our vocabulary complete.")


Initialization of the word vectors and biases for the 24281 tokens in our vocabulary complete.


In [21]:
def find_weight( main_token, context_token ):
  if (context_token,main_token) not in co_occurence_matrix:
    return 0
  if ( co_occurence_matrix[(context_token,main_token)] < x_max_glove_model ):
    return (co_occurence_matrix[(context_token,main_token)] / x_max_glove_model) ** alpha_glove_model
  return 1
alpha_glove_model = 0.75
x_max_glove_model = 100

import numpy as np
import math
def run_single_iteration():
  total_cost = 0
  for (context_token,main_token), value in co_occurence_matrix.items():
    if main_token == context_token:
      continue
    weight = find_weight( main_token, context_token )
    
    if(weight == 0):
      continue
    cost_without_weight = ( np.dot(vectors_main_word[main_token] , vectors_context_word[context_token] ) + biases_main_word[main_token] + biases_context_word[context_token] - math.log(co_occurence_matrix[(context_token,main_token)]))
    total_cost += 0.5 * weight * cost_without_weight ** 2
    gradient_main_word_vector = weight * cost_without_weight * vectors_context_word[context_token]
    gradient_context_word_vector = weight * cost_without_weight * vectors_main_word[main_token]
    gradient_main_bias = weight * cost_without_weight
    gradient_context_bias = weight * cost_without_weight

    vectors_main_word[ main_token ] -= learning_rate * gradient_main_word_vector
    vectors_context_word[ context_token ] -= learning_rate * gradient_context_word_vector

    biases_main_word[ main_token ] -= learning_rate * gradient_main_bias
    biases_context_word[ context_token ] -= learning_rate * gradient_context_bias
  return total_cost

print("Function to run single iteration of gradient descent compiled successfully..!!")

Function to run single iteration of gradient descent compiled successfully..!!


In [25]:
import pickle

learning_rate = 0.03
print("Applying gradient descent to find the appropriate word vectors to carry out unsupervised learning...")
for iteration in range(1,51):
  cost = run_single_iteration()
  print("Iteration " + str(iteration) + " successfull. Returned cost value is: " + str(cost))

print("All iterations fot gradient descent completed successfully..!!")
print("Saving word vectors in file 'word_vectors' ")
pickle.dump(vectors_main_word , open( "/content/drive/My Drive/ISI_PROJECT/word_vectors_test_set_50_iterations_Greater_Alpha", "wb" ) )


Applying gradient descent to find the appropriate word vectors to carry out unsupervised learning...
Iteration 1 successfull. Returned cost value is: 2341621.1353613264
Iteration 2 successfull. Returned cost value is: 548151.0081366764
Iteration 3 successfull. Returned cost value is: 352049.8694298474
Iteration 4 successfull. Returned cost value is: 273242.3247712233
Iteration 5 successfull. Returned cost value is: 228303.79072655636
Iteration 6 successfull. Returned cost value is: 198739.52547601215
Iteration 7 successfull. Returned cost value is: 177609.22943605023
Iteration 8 successfull. Returned cost value is: 161663.4733580541
Iteration 9 successfull. Returned cost value is: 149160.51153276142
Iteration 10 successfull. Returned cost value is: 139074.39796444552
Iteration 11 successfull. Returned cost value is: 130756.88484217756
Iteration 12 successfull. Returned cost value is: 123775.7269185333
Iteration 13 successfull. Returned cost value is: 117830.32062513557
Iteration 14 suc

In [26]:
import pickle

learning_rate = 0.01
print("Applying gradient descent to find the appropriate word vectors to carry out unsupervised learning...")
for iteration in range(1,51):
  cost = run_single_iteration()
  print("Iteration " + str(50 + iteration) + " successfull. Returned cost value is: " + str(cost))

print("All iterations fot gradient descent completed successfully..!!")
print("Saving word vectors in file 'word_vectors' ")
pickle.dump(vectors_main_word , open( "/content/drive/My Drive/ISI_PROJECT/word_vectors_test_set_100_iterations_Greater_Alpha", "wb" ) )


Applying gradient descent to find the appropriate word vectors to carry out unsupervised learning...
Iteration 51 successfull. Returned cost value is: 69770.7293682521
Iteration 52 successfull. Returned cost value is: 64045.99693202178
Iteration 53 successfull. Returned cost value is: 63635.52330389594
Iteration 54 successfull. Returned cost value is: 63355.743858315946
Iteration 55 successfull. Returned cost value is: 63123.06645716154
Iteration 56 successfull. Returned cost value is: 62913.417968003654
Iteration 57 successfull. Returned cost value is: 62717.91939923795
Iteration 58 successfull. Returned cost value is: 62532.4386077651
Iteration 59 successfull. Returned cost value is: 62354.68038837473
Iteration 60 successfull. Returned cost value is: 62183.204129496655
Iteration 61 successfull. Returned cost value is: 62017.027757457174
Iteration 62 successfull. Returned cost value is: 61855.44234478261
Iteration 63 successfull. Returned cost value is: 61697.914402154696
Iteration 64

In [27]:
import pickle

learning_rate = 0.01
print("Applying gradient descent to find the appropriate word vectors to carry out unsupervised learning...")
for iteration in range(1,51):
  cost = run_single_iteration()
  print("Iteration " + str(100 + iteration) + " successfull. Returned cost value is: " + str(cost))

print("All iterations fot gradient descent completed successfully..!!")
print("Saving word vectors in file 'word_vectors' ")
pickle.dump(vectors_main_word , open( "/content/drive/My Drive/ISI_PROJECT/word_vectors_test_set_150_iterations_Greater_Alpha", "wb" ) )


Applying gradient descent to find the appropriate word vectors to carry out unsupervised learning...
Iteration 101 successfull. Returned cost value is: 57221.3562952766
Iteration 102 successfull. Returned cost value is: 57128.7972222028
Iteration 103 successfull. Returned cost value is: 57037.1734871099
Iteration 104 successfull. Returned cost value is: 56946.46880915079
Iteration 105 successfull. Returned cost value is: 56856.66739523306
Iteration 106 successfull. Returned cost value is: 56767.75391530825
Iteration 107 successfull. Returned cost value is: 56679.71347964514
Iteration 108 successfull. Returned cost value is: 56592.53161767311
Iteration 109 successfull. Returned cost value is: 56506.19425838898
Iteration 110 successfull. Returned cost value is: 56420.687712136045
Iteration 111 successfull. Returned cost value is: 56335.99865368336
Iteration 112 successfull. Returned cost value is: 56252.11410632933
Iteration 113 successfull. Returned cost value is: 56169.021427155065
Ite

In [28]:
import pickle

learning_rate = 0.01
print("Applying gradient descent to find the appropriate word vectors to carry out unsupervised learning...")
for iteration in range(1,51):
  cost = run_single_iteration()
  print("Iteration " + str(150 + iteration) + " successfull. Returned cost value is: " + str(cost))

print("All iterations fot gradient descent completed successfully..!!")
print("Saving word vectors in file 'word_vectors' ")
pickle.dump(vectors_main_word , open( "/content/drive/My Drive/ISI_PROJECT/word_vectors_test_set_200_iterations_Greater_Alpha", "wb" ) )


Applying gradient descent to find the appropriate word vectors to carry out unsupervised learning...
Iteration 151 successfull. Returned cost value is: 53498.56728267199
Iteration 152 successfull. Returned cost value is: 53438.9907893634
Iteration 153 successfull. Returned cost value is: 53379.87434385702
Iteration 154 successfull. Returned cost value is: 53321.212485351236
Iteration 155 successfull. Returned cost value is: 53262.99984511786
Iteration 156 successfull. Returned cost value is: 53205.23114435224
Iteration 157 successfull. Returned cost value is: 53147.90119217313
Iteration 158 successfull. Returned cost value is: 53091.00488353104
Iteration 159 successfull. Returned cost value is: 53034.537197356796
Iteration 160 successfull. Returned cost value is: 52978.4931946207
Iteration 161 successfull. Returned cost value is: 52922.86801654883
Iteration 162 successfull. Returned cost value is: 52867.65688280826
Iteration 163 successfull. Returned cost value is: 52812.855089872966
I

In [13]:
pickle.dump(vectors_main_word , open( "/content/drive/My Drive/ISI_PROJECT/word_vectors_test_set_50_iterations", "wb" ) )
